In [1]:
'''
This contains code for generating the table "Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time t, sparsity s, L1-Norm 𝛿1, L2-Norm 𝛿2, and validity %. (*) denotes datasets where RFOCSE necessitated uncapped explanation time."

A CSV and TEX version will be generated, adjustments to the LaTeX table fontsize and table width may be neccessary

Experiment results files needed: CompareMethods

Results used in the paper are provided in "../results/final" if generating new results run each experiment and update the results paths below
'''
# path to each result file
results_path = "../results/compare_methods_tab3.csv"

# path to output the figure
export_figures = True
output_dir = "./reproducibility/"
table_save_name = "compare_methods_tab3"

In [2]:
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath("../"))
#from dataset import DS_NAMES
#from utilities.figure_tools import (get_latest_results_directory, load_results,
                                    #make_fig_directory)

cmap = plt.get_cmap("Set1")
colors = cmap.colors
if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [3]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 10]
all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()

found_expl = list(all_results["explainer"].unique())
found_ds = list(all_results["dataset"].unique())
print("explainers:", found_expl)
print("datasets", found_ds)
all_results.head()
ds_order = ["compas", "glass", "vertebral"]
expl_order = ["FACET", "MACE", "OCEAN", "RFOCSE", "AFT"]

explainers: ['AFT', 'FACET', 'OCEAN']
datasets ['adult', 'cancer', 'compas', 'credit', 'glass', 'magic', 'spambase', 'vertebral']


In [4]:
opt_vals = {}
bold_opt = False
domin = {
    "sample_time": True,
    "avg_length": True,
    "avg_dist": True,
    "avg_manhattan": True,
    "per_valid": False,
}
metrics = ["sample_time", "avg_length", "avg_dist", "avg_manhattan", "per_valid"]


for ds in all_results["dataset"].unique():
    idx = (all_results["dataset"] == ds)
    opt_vals[ds] = {}
    for m in metrics:
        if domin[m]:
            opt_vals[ds][m] = all_results[idx][m].min()
        else:
            opt_vals[ds][m] = all_results[idx][m].max()

metric_latex = {
    "sample_time": "$t\downarrow$",
    "avg_length": "$\delta_0\downarrow$",
    "avg_manhattan": "$\delta_1\downarrow$",
    "avg_dist": "$\delta_2\downarrow$",
    "per_valid": "$\%\\uparrow$",
}
pretty_names = {
    "FACET": "FACET",
    "MACE": "MACE",
    "AFT": "AFT",
    "OCEAN": "OCEAN",
    "RFOCSE": "RFOCSE",
}
all_metrics = ["sample_time", "avg_length", "avg_manhattan", "avg_dist", "per_valid"]

In [5]:
def df_to_csv_latex(df_source, expls, metrics, fname, include_row_label=True):
    #df_ds = df_source["dataset"].unique()
    df_ds = ["compas", "glass", "vertebral"]
    df = df_source.copy()
    df = df.set_index(["dataset", "explainer"])
    with open(output_dir + fname + ".csv", "w") as csv:
        with open(output_dir + fname + ".tex", "w") as tab:
            # csv header row start
            if include_row_label:
                csv.write("dataset,")
            # tab header row start
            tab.write("\\begin{table*}[t]\n\small\n\centering\n\\begin{tabularx}{0.95\\textwidth}{")
            if include_row_label:
                tab.write("|X")
            tab.write("|")
            for expl in expls:
                for m in metrics:
                    tab.write("c")
                tab.write("|")
            tab.write("}\n\hline")
            if include_row_label:
                tab.write("\\textbf{Dataset}")
            # csv and tab header row
            for expl in expls:
                tab.write(" & \multicolumn{5}{c|}{\\textbf{" + pretty_names[expl] +"}}")
                for m in metrics:
                    csv.write(pretty_names[expl] + ",")
                csv.write(",")
            csv.write("\n")
            tab.write(" " + re.escape("\\") + "\n")
            # header row two
            csv.write(",")
            for expl in expls:
                for m in metrics:
                    csv.write(m + ",")
                    tab.write("& " + metric_latex[m] + " ")
                csv.write(",")
            csv.write("\n")
            tab.write(re.escape("\\") + "\n\hline\n")
            # csv and tab body row
            for ds in df_ds:
                if include_row_label:
                    csv.write(ds + ",")
                    tab.write(ds)
                for expl in expls:
                    for m in metrics:
                        if m == "per_valid":
                            val = df.loc[ds, expl][m] * 100
                            val_str = "{:0.1f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_length":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_dist":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACET"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_manhattan":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACET"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "sample_time":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.4f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        else:
                            val_str = str(df.loc[ds, expl][m])
                            csv.write(val_str + ",")
                            tab.write(" & " + val_str)
                    csv.write(",")
                csv.write("\n")
                tab.write(" " + re.escape("\\") + "\n")
            # tab latex close
            tab.write("\hline\n\end{tabularx}\n")
            tab.write("\caption{Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time $t$, explanation distance $\delta$, and percent of instances successfully explained. ($\\ast$) denotes cases which necessitated uncapped explanation time.}\n")
            tab.write("\label{tab.compare_methods}\n\\vspace{-7mm}\n\end{table*}\n")

In [6]:
def df_to_csv_latex_transponse(df_source, expls, ds_names, metrics, fname, include_row_label=True):
    valid_expls = []
    for e in expls: 
        if e in found_expl:
            valid_expls.append(e)
    valid_expls
    print(expls)

    valid_ds = []
    for ds in ds_names: 
        if ds in found_ds:
            valid_ds.append(ds)
    ds_names = valid_ds
    print(ds_names)

    df_ds = df_source["dataset"].unique()
    df = df_source.copy()
    df = df.set_index(["dataset", "explainer"])

    with open(output_dir + fname + ".csv", "w") as csv:
        with open(output_dir + fname + ".tex", "w") as tab:
            # csv header row start
            if include_row_label:
                csv.write("dataset,")
            # tab header row start
            tab.write("\\begin{table*}[t]\n\small\n\centering\n\\begin{tabularx}{0.95\\textwidth}{")
            if include_row_label:
                tab.write("|X")
            tab.write("|")
            for ds in valid_ds:
                for m in metrics:
                    tab.write("c")
                tab.write("|")
            tab.write("}\n\hline")
            if include_row_label:
                tab.write("\\textbf{Dataset}")
            # csv and tab header row
            for ds in valid_ds:
                tab.write(" & \multicolumn{5}{c|}{\\textbf{" + ds.upper() +"}}")
                for m in metrics:
                    csv.write(ds.upper() + ",")
                csv.write(",")
            csv.write("\n")
            tab.write(" " + re.escape("\\") + "\n")
            # header row two
            csv.write(",")
            for ds in valid_ds:
                for m in metrics:
                    csv.write(m + ",")
                    tab.write("& " + metric_latex[m] + " ")
                csv.write(",")
            csv.write("\n")
            tab.write(re.escape("\\") + "\n\hline\n")
            # csv and tab body row
            for expl in valid_expls:
                if include_row_label:
                    csv.write(pretty_names[expl] + ",")
                    tab.write(pretty_names[expl])
                for ds in valid_ds:
                    for m in metrics:
                        if m == "per_valid":
                            val = df.loc[ds, expl][m] * 100
                            val_str = "{:0.1f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_length":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_dist":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACET"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_manhattan":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACET"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "sample_time":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.4f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        else:
                            val_str = str(df.loc[ds, expl][m])
                            csv.write(val_str + ",")
                            tab.write(" & " + val_str)
                    csv.write(",")
                csv.write("\n")
                tab.write(" " + re.escape("\\") + "\n")
            # tab latex close
            tab.write("\hline\n\end{tabularx}\n")
            tab.write("\caption{Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time $t$, explanation distance $\delta$, and percent of instances successfully explained. ($\\ast$) denotes cases which necessitated uncapped explanation time.}\n")
            tab.write("\label{tab.compare_methods}\n\\vspace{-7mm}\n\end{table*}\n")

In [7]:
# create the table from the main paper
main_paper_ds = ["adult", "cancer", "credit", "magic", "spambase"]
output_expl = [_ for _ in expl_order if _ in found_expl]
df_to_csv_latex_transponse(all_results, output_expl, main_paper_ds, all_metrics, table_save_name, True)

['FACET', 'OCEAN', 'AFT']
['adult', 'cancer', 'credit', 'magic', 'spambase']


In [8]:
# create the table from the appendix
apdx_ds = ["compas", "glass", "vertebral"]
output_expl = [_ for _ in expl_order if _ in found_expl]
df_to_csv_latex_transponse(all_results, output_expl, apdx_ds, all_metrics, table_save_name + "_apdx", True)

['FACET', 'OCEAN', 'AFT']
['compas', 'glass', 'vertebral']


In [9]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["per_valid"])

per_valid            
explainer       AFT FACET OCEAN
dataset                        
adult          0.90   1.0   1.0
cancer         0.35   1.0   1.0
compas         1.00   1.0   1.0
credit         1.00   1.0   1.0
glass          1.00   1.0   1.0
magic          0.95   1.0   1.0
spambase       0.85   1.0   1.0
vertebral      1.00   1.0   1.0

In [10]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

avg_dist                    
explainer       AFT     FACET     OCEAN
dataset                                
adult      0.077331  0.095459  0.445999
cancer     0.199491  0.144914  0.144029
compas     0.130014  0.213750  0.559843
credit     0.437774  0.349907  0.634657
glass      0.132761  0.129994  0.115248
magic      0.139940  0.129426  0.162168
spambase   0.042308  0.007987  0.004213
vertebral  0.076872  0.058838  0.063446

In [11]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["sample_time"])

sample_time                    
explainer         AFT     FACET     OCEAN
dataset                                  
adult        0.002501  0.173339  0.451600
cancer       0.001650  0.069250  0.738099
compas       0.003100  0.002400  0.711644
credit       0.002200  0.284840  1.640839
glass        0.001700  0.004650  1.023644
magic        0.002800  0.002250  2.096495
spambase     0.002550  0.164485  0.472220
vertebral    0.001700  0.000951  0.945135